In [1]:
import pandas            as pd
import numpy             as np
import seaborn           as sb
import scipy.stats       as sp
import matplotlib.pyplot as plt
from sklearn.preprocessing import RobustScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

import tensorflow as tf
from tensorflow import keras


In [2]:
tf.random.set_seed(1)

NumOfData=pd.read_csv('./AmountOfDataAll',sep=',')
NumOfData=NumOfData.set_index('Unnamed: 0')
NumOfData

,test_normal,test_abnormal,train_normal,train_abnormal
Unnamed: 0,,,,
mode2,38,29,19,22
mode3,40,28,166,683


In [3]:
##데이터 불러와서 합치기

#train_normal_mode3
for i in range(int(NumOfData['train_normal'].iloc[1])):
    s6="train_normal_mode3_%d=pd.read_csv('./train_normal_mode3_%d',sep=',')" %(i+1,i+1)
    exec(s6)
print(i+1)

#train_abnormal_mode3
for i in range(int(NumOfData['train_abnormal'].iloc[1])):
    s8="train_abnormal_mode3_%d=pd.read_csv('./train_abnormal_mode3_%d',sep=',')" %(i+1,i+1)
    exec(s8)
print(i+1)
#166, 683

166
683


In [4]:
#전부 합침 train normal mode3

for i in range(int(NumOfData['train_normal'].iloc[0])):
    if i==0:
        train_normal_mode3_all=train_normal_mode3_1
        
    else:
        s1='train_normal_mode3_all=pd.concat([train_normal_mode3_all, train_normal_mode3_%d])'%(i+1)
        exec(s1)
        
print(train_normal_mode3_all)

#분석을 해보니 시계열 분석은 크게 의미가 없다고 판단, 변수간 관계를 알아보려함.
#normal 전부 합침(mode3,train)

for i in range(int(NumOfData['train_abnormal'].iloc[0])):
    if i==0:
        train_abnormal_mode3_all=train_abnormal_mode3_1
        
    else:
        s1='train_abnormal_mode3_all=pd.concat([train_abnormal_mode3_all, train_abnormal_mode3_%d])'%(i+1)
        exec(s1)
        
print(train_abnormal_mode3_all)
#42229
#50649

       Air temp, TC inlet (TE29)  Base duration gas admission  \
0                      -0.042941                    -2.536335   
1                      -0.042405                    -2.536335   
2                      -0.041868                    -2.536335   
3                      -0.041331                    -2.536335   
4                      -0.040794                    -2.536335   
...                          ...                          ...   
10013                   0.199678                     0.505505   
10014                   0.199678                     0.504527   
10015                   0.199678                     0.503548   
10016                   0.199678                     0.502569   
10017                   0.199678                     0.501590   

       Charge Air press, Engine Inlet (PT21X)  \
0                                   -2.947291   
1                                   -2.947182   
2                                   -2.947072   
3                      

In [5]:
#노말 앱노말 비율 맞춤
train_abnormal_mode3_all=train_abnormal_mode3_all.sample(frac=0.84, random_state=1)

In [6]:
print(train_abnormal_mode3_all)

      Air temp, TC inlet (TE29)  Base duration gas admission  \
1662                   1.496511                     0.216785   
3558                  -0.471283                     0.354783   
7036                  -0.470746                     0.302178   
2974                   0.903918                     0.198923   
5876                   1.232421                     0.098361   
...                         ...                          ...   
5920                   1.295760                     0.184977   
121                    1.592056                     0.056765   
2442                  -0.774020                    -0.637632   
94                     1.646269                     0.200391   
1438                  -0.461621                     0.354539   

      Charge Air press, Engine Inlet (PT21X)  \
1662                                0.237848   
3558                                0.310761   
7036                                0.313141   
2974                                0.1

In [7]:
#normal/abnormal 합침, 섞음
train_mode3_dataset=pd.concat([train_abnormal_mode3_all, train_normal_mode3_all])

train_mode3_dataset=train_mode3_dataset.sample(frac=1.0, random_state=3)

In [8]:
print(train_mode3_dataset)

      Air temp, TC inlet (TE29)  Base duration gas admission  \
525                   -1.522813                    -0.653046   
1534                  -0.444444                    -2.989234   
8215                   0.260333                     0.489356   
3082                   0.913043                     0.085882   
5667                  -0.070317                     0.259848   
...                         ...                          ...   
138                   -0.171766                    -1.411304   
2264                  -0.044015                    -0.075361   
2293                   1.066559                     0.410570   
1822                  -0.470746                    -2.985319   
2498                   0.611916                     0.520920   

      Charge Air press, Engine Inlet (PT21X)  \
525                                -0.843485   
1534                               -3.538799   
8215                                0.223646   
3082                                0.0

In [9]:
for i in range(len(train_mode3_dataset)):
    if train_mode3_dataset.iloc[i,28]=='normal':
        train_mode3_dataset.iloc[i,28]=1
    elif train_mode3_dataset.iloc[i,28]=='abnormal':
        train_mode3_dataset.iloc[i,28]=0
        
print(train_mode3_dataset)


      Air temp, TC inlet (TE29)  Base duration gas admission  \
525                   -1.522813                    -0.653046   
1534                  -0.444444                    -2.989234   
8215                   0.260333                     0.489356   
3082                   0.913043                     0.085882   
5667                  -0.070317                     0.259848   
...                         ...                          ...   
138                   -0.171766                    -1.411304   
2264                  -0.044015                    -0.075361   
2293                   1.066559                     0.410570   
1822                  -0.470746                    -2.985319   
2498                   0.611916                     0.520920   

      Charge Air press, Engine Inlet (PT21X)  \
525                                -0.843485   
1534                               -3.538799   
8215                                0.223646   
3082                                0.0

In [10]:
x=train_mode3_dataset.pop("Fuel mode selected")
x=train_mode3_dataset.pop("dateTime")
print(train_mode3_dataset)

      Air temp, TC inlet (TE29)  Base duration gas admission  \
525                   -1.522813                    -0.653046   
1534                  -0.444444                    -2.989234   
8215                   0.260333                     0.489356   
3082                   0.913043                     0.085882   
5667                  -0.070317                     0.259848   
...                         ...                          ...   
138                   -0.171766                    -1.411304   
2264                  -0.044015                    -0.075361   
2293                   1.066559                     0.410570   
1822                  -0.470746                    -2.985319   
2498                   0.611916                     0.520920   

      Charge Air press, Engine Inlet (PT21X)  \
525                                -0.843485   
1534                               -3.538799   
8215                                0.223646   
3082                                0.0

In [11]:
train_mode3_dataset.describe()

,"Air temp, TC inlet (TE29)",Base duration gas admission,"Charge Air press, Engine Inlet (PT21X)","Charge Air temp, Engine Inlet (TE21X)",DVT control air press (PT43),Engine Load percent,Engine Speed 1 (SE11),Exh. Gas temp average,"Exh. Gas temp, TC Inlet","Exh. Gas temp, TC Outlet",...,"LT CW press, Air Cooler Inlet (PT71)","LT CW temp, Air Cooler Inlet (TE71)","LT CW temp, Air Cooler Outlet (TE72)",Main bearing temp average,Pmax average,TC speed,Waste Gate Valve position command (CV26),(Load-b)/(IMEP+1),(Pmax average-b)/(IMEP+1),Load/(IMEP+1 x Engine speed+1)
count,84774.000000,84774.000000,84774.000000,84774.000000,84774.000000,84774.000000,84774.000000,84774.000000,84774.000000,84774.000000,...,84774.0,84774.000000,84774.000000,84774.000000,84774.000000,84774.000000,84774.000000,84774.000000,84774.000000,84774.000000
mean,0.341313,-0.205400,-0.309976,0.074095,0.333540,-0.092384,-0.032273,-0.179951,0.029798,-0.005942,...,0.0,-0.118144,-0.210035,-0.209881,0.044867,-0.308442,0.232962,0.275364,0.184234,-0.076147
std,0.742159,0.872780,1.033995,0.295824,0.675644,0.781552,0.890620,0.688683,1.214280,1.491689,...,0.0,0.449487,0.551579,1.043964,0.777314,1.234901,1.235937,0.296468,0.930769,0.772653
min,-1.538916,-3.565207,-3.906698,-0.729167,-0.077224,-2.714111,-3.357143,-2.603208,-4.522379,-3.766396,...,0.0,-1.400137,-2.261860,-8.003833,-2.253594,-5.235665,-3.807824,-0.636348,-1.124635,-3.260374
25%,-0.128824,-0.638855,-0.843924,-0.012500,-0.011179,-0.669847,-0.542857,-0.570164,-0.597184,-0.987508,...,0.0,-0.414501,-0.486717,-0.355019,-0.344317,-0.812459,-0.187286,0.053586,-0.751601,-0.400047
50%,0.199678,0.190360,0.134078,-0.012500,0.095413,0.067007,0.000000,-0.004905,0.106745,0.226109,...,0.0,-0.127223,-0.186907,-0.164094,-0.029714,0.213293,0.243032,0.269014,-0.038681,0.094281
75%,0.946860,0.350135,0.278953,0.044167,0.405936,0.656091,0.457143,0.175660,1.104224,0.656035,...,0.0,0.149111,-0.020873,0.565834,0.775543,0.311660,0.974572,0.515502,0.961934,0.502373
max,1.745035,0.645217,0.959407,0.989167,3.315078,0.666186,8.071429,1.019223,1.820130,4.045753,...,0.0,2.043776,2.320683,1.063193,1.259072,1.085966,2.228362,0.938058,3.023944,0.733887


In [12]:
#TC와 Combustion으로 분리
train_mode3_TC=pd.DataFrame.copy(train_mode3_dataset)
train_mode3_Comb=pd.DataFrame.copy(train_mode3_dataset)



print(train_mode3_TC)

      Air temp, TC inlet (TE29)  Base duration gas admission  \
525                   -1.522813                    -0.653046   
1534                  -0.444444                    -2.989234   
8215                   0.260333                     0.489356   
3082                   0.913043                     0.085882   
5667                  -0.070317                     0.259848   
...                         ...                          ...   
138                   -0.171766                    -1.411304   
2264                  -0.044015                    -0.075361   
2293                   1.066559                     0.410570   
1822                  -0.470746                    -2.985319   
2498                   0.611916                     0.520920   

      Charge Air press, Engine Inlet (PT21X)  \
525                                -0.843485   
1534                               -3.538799   
8215                                0.223646   
3082                                0.0

In [14]:
print(train_mode3_Comb)

      Air temp, TC inlet (TE29)  Base duration gas admission  \
525                   -1.522813                    -0.653046   
1534                  -0.444444                    -2.989234   
8215                   0.260333                     0.489356   
3082                   0.913043                     0.085882   
5667                  -0.070317                     0.259848   
...                         ...                          ...   
138                   -0.171766                    -1.411304   
2264                  -0.044015                    -0.075361   
2293                   1.066559                     0.410570   
1822                  -0.470746                    -2.985319   
2498                   0.611916                     0.520920   

      Charge Air press, Engine Inlet (PT21X)  \
525                                -0.843485   
1534                               -3.538799   
8215                                0.223646   
3082                                0.0

In [26]:
#train_mode3_Comb.pop("Air temp, TC inlet (TE29)")

#train_mode3_Comb.pop("LT CW press, Air Cooler Inlet (PT71)")
#train_mode3_Comb.pop("LT CW temp, Air Cooler Inlet (TE71)")
#train_mode3_Comb.pop("TC speed")

#train_mode3_Comb.pop("Charge Air press, Engine Inlet (PT21X)") 
##train_mode3_Comb.pop("Charge Air press, Engine Inlet (PT21X)") 

##train_mode3_Comb.pop("Charge Air temp, Engine Inlet (TE21X)")

#train_mode3_Comb.pop("Exh. Gas temp, TC Outlet")

#train_mode3_Comb.pop("LT CW temp, Air Cooler Inlet (TE71)")

#train_mode3_Comb.pop("HT CW temp, Engine Inlet (TE75)") 
#train_mode3_Comb.pop("HT CW temp, Engine Outlet (TE76)")
#train_mode3_Comb.pop("LT CW temp, Air Cooler Outlet (TE72)")
    
##train_mode3_Comb.pop("Main bearing temp average")

##train_mode3_Comb.pop("dateTime")
print(train_mode3_Comb)

      Air temp, TC inlet (TE29)  Base duration gas admission  \
525                   -1.522813                    -0.653046   
1534                  -0.444444                    -2.989234   
8215                   0.260333                     0.489356   
3082                   0.913043                     0.085882   
5667                  -0.070317                     0.259848   
...                         ...                          ...   
138                   -0.171766                    -1.411304   
2264                  -0.044015                    -0.075361   
2293                   1.066559                     0.410570   
1822                  -0.470746                    -2.985319   
2498                   0.611916                     0.520920   

      Charge Air press, Engine Inlet (PT21X)  \
525                                -0.843485   
1534                               -3.538799   
8215                                0.223646   
3082                                0.0

In [30]:
for col in train_mode3_Comb.columns:
    if col == "TC speed" or col == "LT CW press, Air Cooler Inlet (PT71)" or col=="Charge Air temp, Engine Inlet (TE21X)" or col=="Charge Air press, Engine Inlet (PT21X)" or col=="Air temp, TC inlet (TE29)":
        del train_mode3_Comb[col]
print(train_mode3_Comb)

      Base duration gas admission  DVT control air press (PT43)  \
525                     -0.653046                      3.143900   
1534                    -2.989234                     -0.042637   
8215                     0.489356                     -0.001681   
3082                     0.085882                      0.404266   
5667                     0.259848                     -0.060517   
...                           ...                           ...   
138                     -1.411304                     -0.019240   
2264                    -0.075361                      0.022059   
2293                     0.410570                      0.035184   
1822                    -2.985319                     -0.038789   
2498                     0.520920                     -0.017979   

      Engine Load percent  Engine Speed 1 (SE11)  Exh. Gas temp average  \
525             -0.673286               0.014286              -0.769985   
1534            -2.344131               0.300

In [35]:
for col in train_mode3_TC.columns:
    if col == "LT CW temp, Air Cooler Inlet (TE71)" or col == "LT CW press, Air Cooler Inlet (PT71)" or col=="Charge Air temp, Engine Inlet (TE21X)" or col=="Charge Air press, Engine Inlet (PT21X)" or col=="Engine Load percent":
        del train_mode3_TC[col]
    if col == "Base duration gas admission" or col == "DVT control air press (PT43)" or col=="Engine Speed 1 (SE11)" or col=="Exh. Gas temp average":
        del train_mode3_TC[col]
    if col=="FO Pressure Engine Inlet" or col=="FO Temperature Engine Inlet":
        del train_mode3_TC[col]
    if col == "Gas press, Engine Inlet (PT87)" or col == "HT CW temp, Engine Outlet (TE76)" or col=="IMEP average" or col=="LT CW temp, Air Cooler Outlet (TE72)":
        del train_mode3_TC[col]
    if col == "Pmax average" or col == "Waste Gate Valve position command (CV26)" or col=="(Load-b)/(IMEP+1)" or col=="(Pmax average-b)/(IMEP+1)" or col=="Load/(IMEP+1 x Engine speed+1)":
        del train_mode3_TC[col]
    if col=="Gas temp (TE81)" or col=="HT CW temp, Engine Inlet (TE75)":
        del train_mode3_TC[col]


print(train_mode3_TC)

      Air temp, TC inlet (TE29)  Exh. Gas temp, TC Inlet  \
525                   -1.522813                -0.250473   
1534                  -0.444444                -2.319185   
8215                   0.260333                 0.075226   
3082                   0.913043                 1.046018   
5667                  -0.070317                -0.566716   
...                         ...                      ...   
138                   -0.171766                -1.113889   
2264                  -0.044015                -1.098760   
2293                   1.066559                 0.161168   
1822                  -0.470746                -2.395881   
2498                   0.611916                -1.107796   

      Exh. Gas temp, TC Outlet  TC speed state  
525                   0.225172 -0.815424     0  
1534                  3.814647 -4.565864     1  
8215                 -0.551062  0.326216     1  
3082                  0.236883  0.116072     0  
5667                 -1.181605  0.

In [36]:
train_mode3_TC.to_csv('train_mode3_TC.csv')
train_mode3_Comb.to_csv('train_mode3_Comb.csv')